In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [8]:
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

In [9]:
data_dir = '../input/'
df_seeds = pd.read_csv(data_dir + 'NCAATourneySeeds.csv')
df_tour = pd.read_csv(data_dir + 'NCAATourneyCompactResults.csv')

In [10]:
def seed_to_int(seed):
    s_int = int(seed[1:3])
    return s_int
df_seeds['seed_int'] = df_seeds.Seed.apply(seed_to_int)
df_seeds.drop(labels=['Seed'], inplace=True, axis=1)
df_seeds.head()

In [11]:
df_tour.drop(labels=['DayNum', 'WScore', 'LScore', 'WLoc', 'NumOT'], inplace=True, axis=1)
df_tour.head()

In [12]:
df_winseeds = df_seeds.rename(columns={'TeamID':'WTeamID', 'seed_int':'WSeed'})
df_lossseeds = df_seeds.rename(columns={'TeamID':'LTeamID', 'seed_int':'LSeed'})
df_dummy = pd.merge(left=df_tour, right=df_winseeds, how='left', on=['Season', 'WTeamID'])
df_concat = pd.merge(left=df_dummy, right=df_lossseeds, on=['Season', 'LTeamID'])
df_concat['SeedDiff'] = df_concat.WSeed - df_concat.LSeed
df_concat.head()

In [13]:
df_concat.info()

In [14]:
df_concat2 = df_concat.ix[df_concat['Season']>1986]

In [15]:
df_concat2.head()

In [17]:
df_wins = pd.DataFrame()
df_wins['SeedDiff'] = df_concat2['SeedDiff']
df_wins['Result'] = 1

df_losses = pd.DataFrame()
df_losses['SeedDiff'] = -df_concat2['SeedDiff']
df_losses['Result'] = 0

df_predictions = pd.concat((df_wins, df_losses))
df_predictions.head()

In [28]:
df_predictions2 = df_predictions.ix[df_predictions['SeedDiff']<0]

In [19]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [29]:
fig, ax = plt.subplots(figsize=(12, 6))
sns.barplot(x='SeedDiff',y='Result',data=df_predictions2)

In [31]:
fig, ax = plt.subplots(figsize=(12, 6))
sns.countplot(x='SeedDiff',data=df_predictions2)